# CODE - Bachelor's Thesis - Bachelor of Science in Physics

## Raúl Almuzara

### July 2021

#### Complete Python code used in the project *Machine learning model to quantify the accordance of work output and objectives in a complex business environment* which is available in English in the PDF file next to this notebook. The original project was written in Spanish and then translated, which is the reason for the name of some variables that have been left untouched.

#### This is an end-to-end data science project carried out with Python and QGIS. I present a complete analysis of data provided by a real company related to the management of urban waste collection. The objective is the creation of a goodness metric to determine the quality of the work orders executed by waste collection vehicles.

#### First, I carry out an adequate selection and transformation of the data, as well as a study of the variables of greatest interest that can be constructed. Next, I proceed with the design of machine learning models that allow us to make predictions using classification and regression algorithms based on decision trees and neural networks. For each model, we evaluate its capacity with the relevant statistical techniques and I show the most important results for the understanding of the dataset, as well as the implications of the predictions.

---

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import traj_dist.distance as tdist
import random
import seaborn as sn
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, validation_curve, KFold, GridSearchCV, cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from yellowbrick.classifier import ROCAUC
from xgboost import XGBClassifier, XGBRegressor

## The following function takes the names of a CSV of theoretical routes, a CSV of actual tracks and a CSV of dumpsters. It creates other 3 CSVs with only those work orders which are common to the three original files, as well as removing duplicates.

In [ ]:
def filtra_ordenes(csvteoricas,csvreales,csvcoordenadas):
    dfteoricas=pd.read_csv(csvteoricas,sep=";")
    dfreales=pd.read_csv(csvreales,sep=";")
    dfcoordenadas=pd.read_csv(csvcoordenadas,sep=";")
    
    dfcoordenadas["Latitude"] = dfcoordenadas["Latitude"].astype(str).str.replace(",",".")
    dfcoordenadas["Longitude"] = dfcoordenadas["Longitude"].astype(str).str.replace(",",".")
    
    todasreales = pd.Series(dfreales['WorkOrderId'])
    duplicadas = list(todasreales[todasreales.duplicated()])    
    
    ordenesteoricas=np.unique(dfteoricas['WorkOrderId'])
    ordenesreales = np.array([x for x in dfreales['WorkOrderId'] if x not in duplicadas])
    ordenescoordenadas=np.unique(dfcoordenadas['WorkOrderId'])
    ordenes=[ordenesteoricas,ordenesreales,ordenescoordenadas]
    ordenescomunes=np.sort(list(set.intersection(*map(set,ordenes))))
    
    filtroteorico=dfteoricas[dfteoricas['WorkOrderId'].isin(ordenescomunes)]
    filtroreal=dfreales[dfreales['WorkOrderId'].isin(ordenescomunes)]
    filtrocoord=dfcoordenadas[dfcoordenadas['WorkOrderId'].isin(ordenescomunes)]
    
    filtrocoordfinal = pd.DataFrame(columns=filtrocoord.columns)
    
    for orden in ordenescomunes:
        filtrocoordfinal = filtrocoordfinal.append(filtrocoord[filtrocoord['WorkOrderId']==orden].drop_duplicates(subset=['ContainerId']),ignore_index=True)
        
    filtroteorico.to_csv('FiltroTeorico_'+csvteoricas,sep=';',index=False)
    filtroreal.to_csv('FiltroReal_'+csvreales,sep=';',index=False)
    filtrocoordfinal.to_csv('FiltroCoord_'+csvcoordenadas,sep=';',index=False)

## Densification function (it adds extra equispaced points to the arrays of points that define the trajectories)

In [ ]:
def densificar(coordenadas, segmentos):
    denso=coordenadas
    for i in range(len(coordenadas)-1):
        denso=np.insert(denso,i*segmentos+1,np.array([list(a) for a in zip(np.linspace(coordenadas[i][0],coordenadas[i+1][0],segmentos,endpoint=False)[1:],np.linspace(coordenadas[i][1],coordenadas[i+1][1],segmentos,endpoint=False)[1:])]),0)
    return denso

## Definition of the function that processes the data to obtain relevant variables

In [ ]:
#Secondary function that helps the main function
def clasifica1(cen_x):
    if cen_x==centroides[0]:
        return 0
    if cen_x==centroides[1]:
        return 1
    if cen_x==centroides[2]:
        return 2
    if cen_x==centroides[3]:
        return 3

#Secondary function that helps the main function
def clasifica2(Bondad):
    if Bondad>=0 and Bondad<min1:
        return 0
    if Bondad>=min1 and Bondad<min2:
        return 1
    if Bondad>=min2 and Bondad<min3:
        return 2
    if Bondad>=min3:
        return 3

#Main function that generates all variables.
#It takes 3 CSVs of theoretical routes, actual routes and filtered containers (by the filtering function above).
#It returns a dataframe.
def calcula_variables(csvteoricas,csvreales,csvcoordenadas):

    ordenes = pd.read_csv(csvcoordenadas,sep=';')

    ordenesag = ordenes.groupby('WorkOrderId').agg({
        'Scheduled': 'sum',
        'Collected': 'sum'
    })
    
    #Obtains number of containers of each type in each work order
    ordenesag['NContenedores']=ordenes.groupby('WorkOrderId').count().iloc[:,2]
    ordenesag['01']=ordenesag['NContenedores']-ordenesag['Scheduled']
    ordenesag['10']=ordenesag['NContenedores']-ordenesag['Collected']
    ordenesag['11']=ordenesag['NContenedores']-ordenesag['01']-ordenesag['10']
    ordenesag=ordenesag.drop(columns=['Scheduled','Collected'])
    ordenesag['Ruido'] = None #Column for random noise
    ordenesag['BienHechos']=ordenesag['11']/(ordenesag['11']+ordenesag['10']) #Proportion of well-collected containers
    ordenesag['Similitud'] = None #Degree of similarity
    ordenesag['Adicionales']=ordenesag['01']/(ordenesag['11']+ordenesag['10']+ordenesag['01']) #Proportion of additional containers collected
    ordenesag['RatioLongitudes'] = None #Relative deviation between theoretical and actual length

    teoricas = pd.read_csv(csvteoricas,sep=";")
    reales = pd.read_csv(csvreales,sep=";")

    for j in range(len(reales['WorkOrderId'])):
        wktr = reales['WKT'].iloc[j].replace("LINESTRING(" , "").replace(")","").replace(","," ").split()
        wktreales = []
        
        #Obtains the array of points of the actual route and its length
        for i in range(0,len(wktr),2):
            wktreales.append([float(wktr[i]),float(wktr[i+1])])
        longitudes_r = np.sqrt(np.sum(np.diff(np.array(wktreales), axis=0)**2, axis=1))
        longitud_real = np.sum(longitudes_r)

        wktt = teoricas['WKT'].iloc[j].replace("MULTILINESTRING((" , "").replace("))","").replace(","," ").split()
        wktteoricas = []
        
        #Obtains the array of points of the theoretical route and its length
        for i in range(0,len(wktt),2):
            wktteoricas.append([float(wktt[i]),float(wktt[i+1])])
        longitudes_t = np.sqrt(np.sum(np.diff(np.array(wktteoricas), axis=0)**2, axis=1))
        longitud_teorica = np.sum(longitudes_t)

        ratio = np.abs(longitud_real-longitud_teorica)/longitud_teorica
        ordenesag['RatioLongitudes'].iloc[j]=ratio

        distancia = tdist.sspd(np.array(wktreales),np.array(wktteoricas),'euclidean') #For higher accuracy, use 'spherical' (haversine formula) if the points are pairs (longitude,latitude)
        ordenesag['Similitud'].iloc[j]=distancia
        
        ordenesag['Ruido'].iloc[j]=np.random.normal(0,0.7) #To make the models more realistic and hinder training, increase the standard deviation of the Gaussian noise
    
    ordenesag['Similitud']=0.001/(0.001+ordenesag['Similitud']) #Normalization in the range [0,1]. Parameter must be carefully chosen (0.001)
    
    ordenesag['RatioLongitudes']=(ordenesag['RatioLongitudes']-ordenesag['RatioLongitudes'].min())/(ordenesag['RatioLongitudes'].max()-ordenesag['RatioLongitudes'].min()) #Normalization to range [0,1]
        
    ordenesag['Bondad']=10*ordenesag['BienHechos']+4*ordenesag['Similitud']+2*ordenesag['Adicionales']-1*ordenesag['RatioLongitudes']+ordenesag['Ruido'] #Linear combination with weights 10, 4, 2, -1 plus noise
    
    ordenesag['Bondad']=10*(ordenesag['Bondad']-ordenesag['Bondad'].min())/(ordenesag['Bondad'].max()-ordenesag['Bondad'].min()) #Normalization to range [0,10]
    
    
    #Clustering
    
    mezcla = ordenesag.copy()

    kmeans = KMeans(n_clusters=4, random_state=0)
    mezcla['cluster'] = kmeans.fit_predict(mezcla[['Bondad']])
    centroids = kmeans.cluster_centers_
    cen_x = [i[0] for i in centroids] 
    mezcla['cen_x'] = mezcla.cluster.map({0:cen_x[0], 1:cen_x[1], 2:cen_x[2], 3:cen_x[3]})
    
    global centroides
    
    centroides = np.sort(np.unique(mezcla['cen_x']))
    colors = ['#DF2020', '#81DF20', '#2095DF', '#20DF95']
    mezcla['c'] = mezcla.cluster.map({0:colors[0], 1:colors[1], 2:colors[2], 3:colors[3]})
    plt.scatter(mezcla.Bondad,mezcla.Bondad, c=mezcla.c, alpha = 0.6, s=10)
    ordenesag['Resultado']=mezcla['cen_x'].apply(clasifica1)
    
    global min0
    global min1
    global min2
    global min3
    
    min0 = min(ordenesag[ordenesag['Resultado']==0]['Bondad'])
    min1 = min(ordenesag[ordenesag['Resultado']==1]['Bondad'])
    min2 = min(ordenesag[ordenesag['Resultado']==2]['Bondad'])
    min3 = min(ordenesag[ordenesag['Resultado']==3]['Bondad'])
    
    print(min0)
    print(min1)
    print(min2)
    print(min3)
    
    ordenesag['Bondad']=ordenesag['Bondad'].astype(float).round(0)
    
    ordenesag['Resultado']=ordenesag['Bondad'].apply(clasifica2)
    
    return ordenesag

## Calculate variables (note that each run will give somewhat different results due to random noise)

In [ ]:
variables = calcula_variables('Rutas_teoricas_623_OTs.csv','Rutas_reales_623_OTs.csv','Contenedores_623_OTs.csv')

## Matrix of plots with variables taken in pairs

In [ ]:
sn.pairplot(variables[["BienHechos", "Similitud", "Adicionales", "RatioLongitudes"]].astype(float))

## Decision tree classifier

In [ ]:
X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(max_depth=3,random_state=42)
clf.fit(X_train, y_train)
fig = plt.figure(figsize=(100,80))
tree.plot_tree(clf, feature_names=['BienHechos','Similitud','Adicionales','RatioLongitudes'], class_names=['0','1','2','3'],filled=True)
plt.show()
y_pred=clf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print(acc)

## Confusion matrix for the decision tree

In [ ]:
%matplotlib inline
plt.figure(figsize=(4,3))
sn.heatmap(confusion_matrix(y_test,y_pred),annot=True)
plt.title('Matriz de confusión del árbol')
plt.xlabel('Clase predicha')
plt.ylabel('Clase real')

## Accuracy versus number of trees in the random forest

In [ ]:
scores = []
for k in range(1, 100):
    rfc = RandomForestClassifier(n_estimators=k,max_depth=3,n_jobs=-1,random_state=42)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))
    
%matplotlib inline

plt.plot(range(1, 100), scores)
plt.title('Precisión del bosque aleatorio en función del número de árboles')
plt.xlabel('Número de árboles')
plt.ylabel('Precisión')
plt.show()

## Random forest classifier

In [ ]:
X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=70,max_depth=3,n_jobs=-1,random_state=42)
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print(acc)

## Confusion matrix for the random forest

In [ ]:
%matplotlib inline
plt.figure(figsize=(4,3))
sn.heatmap(confusion_matrix(y_test,y_pred),annot=True)
plt.title('Matriz de confusión del bosque')
plt.xlabel('Clase predicha')
plt.ylabel('Clase real')

## Decision surfaces training random forests

In [ ]:
n_classes = 4
plot_colors = ['darkorange','lime','deepskyblue','fuchsia']
plot_step = 0.01

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
    
    X = np.array(variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']])[:, pair]
    y = np.array(variables['Resultado'])
    
    clf = RandomForestClassifier(n_estimators=70,max_depth=3,n_jobs=-1,random_state=42).fit(X, y)

    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.gist_rainbow)

    plt.xlabel(['BienHechos','Similitud','Adicionales','RatioLongitudes'][pair[0]])
    plt.ylabel(['BienHechos','Similitud','Adicionales','RatioLongitudes'][pair[1]])

    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=['0','1','2','3'][i],
                    cmap=plt.cm.rainbow, edgecolor='black', s=10)

plt.suptitle("Superficies de decisión")
plt.legend(loc='lower right', borderpad=0, handletextpad=0)
plt.axis("tight")

## Feature importances training a random forest

In [ ]:
importance = rf.feature_importances_

for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

plt.bar([x for x in range(len(importance))], importance)
plt.show()

## Applying bootstrapping to a random forest to obtain a 95% confidence interval

In [ ]:
values = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes','Resultado']].astype(float).values

n_iterations = 1000
n_size = int(len(variables) * 1)

stats = list()
for i in range(n_iterations):

    train = resample(values, n_samples=n_size)
    test = np.array([x for x in values if x.tolist() not in train.tolist()])

    model = RandomForestClassifier(n_estimators=70,max_depth=3,n_jobs=-1,random_state=42)
    model.fit(train[:,:-1], train[:,-1])

    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    print(score)
    stats.append(score)

plt.hist(stats)
plt.show()

alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

## Multiclass ROC curves for the random forest

In [ ]:
X = np.array(variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']])
y = np.array(variables['Resultado'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = RandomForestClassifier(n_estimators=70,max_depth=3,n_jobs=-1,random_state=42)
visualizer = ROCAUC(model, micro=False, macro=False, classes=["0", "1", "2", "3"])

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)

custom_viz = visualizer.ax
custom_viz.set_title("Curvas ROC para bosque aleatorio")
custom_viz.set_xlabel("1 - Especificidad")
custom_viz.set_ylabel("Sensibilidad")
custom_viz.legend()
custom_viz.figure.show()

## Random forest regressor

In [ ]:
X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Bondad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf = RandomForestRegressor(n_estimators=70,max_depth=3,random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
rf.score(X_test,y_test)

## Tuning a multilayer perceptron classifier

In [ ]:
X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

mlp = MLPClassifier()
parameter_space = {
    'hidden_layer_sizes': [(4,),(5,),(6,)],
    'activation': ['tanh','logistic','relu'],
    'solver': ['lbfgs'],
    'alpha': [0.01,0.001,0.0001,0.00001],
    'max_iter': [1000],
}
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

print('Mejores parametros:\n', clf.best_params_)

## Confusion matrix for the neural network

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(4,),activation='relu',solver='lbfgs',alpha=0.0001,learning_rate='constant',max_iter=2000)
mlp.fit(X_train,y_train)
print(mlp.score(X_test,y_test))

%matplotlib inline
plt.figure(figsize=(4,3))
sn.heatmap(confusion_matrix(y_test,mlp.predict(X_test)),annot=True)
plt.xlabel('Clase predicha')
plt.ylabel('Clase real')

## Applying bootstrapping to a neural network to obtain a 95% confidence interval

In [ ]:
values = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes','Resultado']].astype(float).values

n_iterations = 1000
n_size = int(len(variables) * 0.5)

stats = list()
for i in range(n_iterations):

    train = resample(values, n_samples=n_size)
    test = np.array([x for x in values if x.tolist() not in train.tolist()])

    model = MLPClassifier(hidden_layer_sizes=(4,),activation='relu',solver='lbfgs',alpha=0.0001,max_iter=2000,learning_rate='constant')
    model.fit(train[:,:-1], train[:,-1])

    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    print(score)
    stats.append(score)

plt.hist(stats)
plt.show()

alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

variables[['BienHechos','Similitud','Adicionales','RatioLongitudes','Resultado']].astype(float).values

## Training a neural network with Adam in TensorFlow

In [ ]:
def get_batch(x_data, y_data, batch_size):
    idxs = np.random.randint(0, len(y_data), batch_size)
    return x_data[idxs,:], y_data[idxs]

def nn_model(x_input, W1, b1, W2, b2):
    x_input = tf.reshape(x_input, (x_input.shape[0], -1))
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)
    x = tf.nn.relu(x)
    logits = tf.add(tf.matmul(x, W2), b2)
    return logits

def loss_fn(logits, labels):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,
                                                                              logits=logits))
    return cross_entropy

epochs = 80
batch_size = 50

X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = tf.Variable(np.array(X_train).astype('float'))
X_test = tf.Variable(np.array(X_test).astype('float'))

W1 = tf.Variable(tf.random.normal([4, 4], stddev=0.03), name='W1')

b1 = tf.Variable(tf.random.normal([4]), name='b1')

W2 = tf.Variable(tf.random.normal([4, 4], stddev=0.03), name='W2')

b2 = tf.Variable(tf.random.normal([4]), name='b2')

optimizer = tf.keras.optimizers.Adam(learning_rate=0.07,beta_1=0.9,beta_2=0.999,epsilon=0.0000001)

total_batch = int(len(y_train) / batch_size)
arrayloss=[]
arrayacc=[]
for epoch in range(epochs):
    avg_loss = 0
    for i in range(total_batch):
        batch_x, batch_y = get_batch(np.array(X_train), np.array(y_train), batch_size=batch_size)

        batch_x = tf.Variable(np.array(batch_x).astype('float'))
        batch_y = tf.Variable(np.array(batch_y).astype('float'))

        batch_y = tf.one_hot(np.array(batch_y).astype('int'),4)
        with tf.GradientTape() as tape:
            logits = nn_model(batch_x, W1, b1, W2, b2)
            loss = loss_fn(logits, batch_y)
        gradients = tape.gradient(loss, [W1, b1, W2, b2])
        optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2]))
        avg_loss += loss / total_batch
    arrayloss.append(avg_loss)
    test_logits = nn_model(X_test, W1, b1, W2, b2)
    max_idxs = tf.argmax(test_logits, axis=1)
    test_acc = np.sum(max_idxs.numpy() == y_test) / len(y_test)
    arrayacc.append(test_acc)
    print(f"Epoca: {epoch + 1}, perdida={avg_loss:.3f}, precision={test_acc*100:.3f}%")

## Loss curve

In [ ]:
plt.plot(range(1,epochs+1),arrayloss,color='red')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida')
plt.show()

## Accuracy curve

In [ ]:
plt.plot(range(1,epochs+1),arrayacc,color='green')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.title('Curva de precisión')
plt.show()

## Multiclass ROC curves for the neural network

In [ ]:
X = np.array(variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']])
y = np.array(variables['Resultado'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = MLPClassifier(hidden_layer_sizes=(4,),activation='relu',solver='lbfgs',alpha=0.0001,learning_rate='constant',max_iter=2000)
visualizer = ROCAUC(model, micro=False, macro=False, classes=["0", "1", "2", "3"])

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)

custom_viz = visualizer.ax
custom_viz.set_title("Curvas ROC para la red neuronal")
custom_viz.set_xlabel("1 - Especificidad")
custom_viz.set_ylabel("Sensibilidad")
custom_viz.legend()
custom_viz.figure.show()

## Neural network regressor

In [ ]:
X = variables[['BienHechos','Similitud','Adicionales','RatioLongitudes']]
y = variables['Bondad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

mlp = MLPRegressor()
parameter_space = {
    'hidden_layer_sizes': [(4,)],
    'activation': ['relu'],
    'solver': ['lbfgs'],
    'alpha': [0.0001],
    'learning_rate': ['constant'],
}
regr = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
regr.score(X_test,y_test)